## 1. Data Load


In [27]:
from datasets import load_dataset

news_dataset = load_dataset('nlpHakdang/aihub-news30k',  data_files={"train":"news_train_1_1.csv", "valid":"news_valid_1_1.csv"}, use_auth_token='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')

Using custom data configuration nlpHakdang___aihub-news30k-1ea472361ab2489f
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___aihub-news30k-1ea472361ab2489f/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
news_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'category', 'media_sub_type', 'media_name', 'char_count', 'publish_date', 'title', 'original', 'article', 'extractive', 'abstract'],
        num_rows: 271093
    })
    valid: Dataset({
        features: ['id', 'category', 'media_sub_type', 'media_name', 'char_count', 'publish_date', 'title', 'original', 'article', 'extractive', 'abstract'],
        num_rows: 30122
    })
})

## 2. Summarize

In [29]:
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-uswpbj7g/kobart_80b8f982e04944aca699e9bfb94dd51d
  Running command git clone --filter=blob:none -q https://github.com/SKT-AI/KoBART /tmp/pip-install-uswpbj7g/kobart_80b8f982e04944aca699e9bfb94dd51d
  Resolved https://github.com/SKT-AI/KoBART to commit 1811d3779cf394872f32073d01c1201c8525bc00
  Preparing metadata (setup.py) ... done
  Using cached torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl (776.8 MB)
  Using cached transformers-4.3.3-py3-none-any.whl (1.9 MB)
  Created wheel for kobart: filename=kobart-0.5.1-py3-none-any.whl size=9563 sha256=40420e875e72d48e7a837e153f97fba8646c8e3d33a4aac3d164d8dc3ceee74f

In [30]:
from kobart import get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
kobart_tokenizer.tokenize("안녕하세요. 한국어 BART 입니다.🤣:)l^o")

using cached model


['▁안녕하', '세요.', '▁한국어', '▁B', 'A', 'R', 'T', '▁입', '니다.', '🤣', ':)', 'l^o']

In [31]:
from transformers import BartModel
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer

kobart_tokenizer = get_kobart_tokenizer()
model = BartModel.from_pretrained(get_pytorch_kobart_model())

inputs = kobart_tokenizer(['안녕하세요.'], return_tensors='pt')
outputs = model(inputs['input_ids'])

using cached model
using cached model


In [32]:
from transformers.models.bart import BartForConditionalGeneration
import torch

def load_model():
    model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model())
    # tokenizer = get_kobart_tokenizer()
    return model

model = load_model()
tokenizer = get_kobart_tokenizer()

using cached model
using cached model


In [33]:
text = "애플이 내년 하반기 출시할 아이폰14(가칭)에 펀치홀(카메라 구멍) 디스플레이를 적용할 것이라는 전망이 나오면서 아이폰 공급망에 포함돼 있는 삼성디스플레이, LG디스플레이, 중국 BOE의 표정이 엇갈린다. 이미 갤럭시 시리즈에 펀치홀 유기발광다이오드(OLED) 패널을 대량 공급한 이력이 있는 삼성디스플레이는 여유로운 분위기가 감지된다. LG디스플레이는 공급 경험은 없지만 펀치홀 디스플레이의 기술 난이도가 높지 않은 것으로 판단한다. 반면 공급 경험이 있지만 디스플레이 기술력이 상대적으로 떨어지는 BOE는 일단 애플이 요구하는 기술 수준을 따라가는 데 주력할 것으로 보인다.17일 폰아레나 등 해외 정보기술(IT) 전문매체 등에 따르면 애플은 내년 출시할 아이폰14 시리즈 고급 모델인 프로의 전면 디자인에서 ‘노치’를 빼고 펀치홀을 적용한다. 노치는 디스플레이 상단의 일정 부분을 카메라 모듈과 센서 등에 할애하는 디자인으로, 생김새가 한자 ‘오목할 요(凹)’자와 유사하다. 이 때문에 ‘M자 탈모’라고도 불린다. 애플은 페이스ID라는 안면인식 보안기술을 위해 노치 디자인을 지난 2018년 출시한 아이폰X(텐)부터 최근 내놓은 아이폰13까지 유지하고 있다."
#text = "삼성바이오로직스가 최근 주가 급등에 힘입어 네이버를 밀어내고 코스피 시가총액 3위에 올랐다. 17일 오전 9시51분 기준 삼성바이오로직스는 96만2,000원에 거래되고 있다. 맥쿼리, CS증권 등 외국계 창구를 통한 순매수세가 유입되고 있다. 삼성바이오로직스는 전일에도 6% 가까이 상승 마감했으며 지난 14일부터 4거래일 연속 상승세를 나타내고 있다. 시가총액은 현재가 기준 63조7,831억 원으로 네이버를 앞질렀다. 삼성바이오로직스와 네이버의 시총 격차는 현재 3,774억 원 수준이다."
if text:
    text = text.replace('\n', '')
    input_ids = tokenizer.encode(text)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0)
    output = model.generate(input_ids, eos_token_id=1, min_length=32, max_length=512, num_beams=5)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    print(output)

RuntimeError: Integer division of tensors using div or / is no longer supported, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.

In [18]:
# 대화 시스템 평가 , 번역의 정답.

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [20]:
import torch
from kobert.pytorch_kobert import get_pytorch_kobert_model
input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
model, vocab  = get_pytorch_kobert_model()
sequence_output, pooled_output = model(input_ids, input_mask, token_type_ids)

using cached model. /opt/ml/final-project-level3-nlp-06/summarize/.cache/kobert_v1.zip
using cached model. /opt/ml/final-project-level3-nlp-06/summarize/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [21]:
pooled_output.shape

torch.Size([2, 768])

In [22]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [23]:
sequence_output[0]

tensor([[-0.2461,  0.2428,  0.2590,  ..., -0.4861, -0.0731,  0.0756],
        [-0.2478,  0.2420,  0.2552,  ..., -0.4877, -0.0727,  0.0754],
        [-0.2472,  0.2420,  0.2561,  ..., -0.4874, -0.0733,  0.0765]],
       grad_fn=<SelectBackward>)

In [24]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-mkn6k4ek/kobert-tokenizer_e5ab57838c9c4671bf68d5cb457322d2
  Running command git clone --filter=blob:none -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-mkn6k4ek/kobert-tokenizer_e5ab57838c9c4671bf68d5cb457322d2
  Resolved https://github.com/SKTBrain/KoBERT.git to commit d0c0829be97b3a363a8035d4131e8be4988e129d
  Preparing metadata (setup.py) ... done


In [25]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [26]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
output = model.generate(inputs, eos_token_id=1, max_length=64, num_beams=5)
print(output)

AttributeError: 

In [ ]:
output = model.generate(input_ids, eos_token_id=1, max_length=64, num_beams=5)
output = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
ARTICLE = "삼성바이오로직스가 최근 주가 급등에 힘입어 네이버를 밀어내고 코스피 시가총액 3위에 올랐다. 17일 오전 9시51분 기준 삼성바이오로직스는 96만2,000원에 거래되고 있다. 맥쿼리, CS증권 등 외국계 창구를 통한 순매수세가 유입되고 있다. 삼성바이오로직스는 전일에도 6% 가까이 상승 마감했으며 지난 14일부터 4거래일 연속 상승세를 나타내고 있다. 시가총액은 현재가 기준 63조7,831억 원으로 네이버를 앞질렀다. 삼성바이오로직스와 네이버의 시총 격차는 현재 3,774억 원 수준이다."


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BertConfig

config = BertConfig.from_pretrained("skt/kobert-base-v1")
model = AutoModelForSeq2SeqLM.from_pretrained("skt/kobert-base-v1")
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0]))

In [ ]:

from transformers import pipeline

# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization")
summarized = summarizer(inputs["input_ids"], min_length=75, max_length=300)

# Print summarized text
print(summarized)